# Python Code Snippets

## Декораторы

Функции в `Python` -  функции высшего порядка. Декораторы - это специальные функции, которые служат для временного изменения поведения других функций. Они принимают в качестве параметра старую функцию и возвращают новую. 

Декораторы позволяют подобный код:

In [1]:
def add(x: int, y: int) -> int:
    return x + y

def plus_one(old_func):
    def new_func(x: int, y: int) -> int:
        return old_func(x, y) + 1
    
    return new_func


new_add = plus_one(add)
new_add(5, 40)

46

Переписать в виде

In [2]:
import functools

def plus_one(old_func):
    @functools.wraps(old_func)
    def wrapper(*args, **kwargs):  
        return old_func(*args, **kwargs) + 1
    return wrapper

@plus_one
def add(x: int, y: int) -> int:
    """Docstring"""    
    return x + y

add(5, 10)

16

Можно обратить внимание, что "новая" декорируемая функция аналогично обернута в декоратор 
```python  
@functools.wraps 
```
это позволяет сохранять метаинформацию (название, `docstring`)

In [3]:
assert add.__name__ == "add"
assert add.__doc__ == "Docstring"

## Контекстные менеджеры

Контекстные менеджеры позволяют контролировать выделение и освобождение ресурсов с помощью специального оператора `with`.  Например для закрытие файлового дескриптора, освобождения блокировки, закрытия соединения с базой данных и т.д.

Пример с файлом:
```python
with open('file.txt', 'w', encoding='utf-8') as f:
    f.write('Hello\n')

```
в заголовке оператора `with` создается объект файлового дескриптора, который автоматически закрывается при выходе из тела оператора.

Контекстный менеджер можно реализовать самостоятельно. Следующий код

In [4]:

class MyContextMng(object):
    def __init__(self, s):
        self.s = s
        pass

    def __enter__(self):   
        print('Enter')
        return self.s

    def __exit__(self, exception_type, exception_val, trace):
        print('Exit')
        return True
    
    
with MyContextMng('Hello') as f:
    #actions
    pass


Enter
Exit


приблизительное преобразуется в следующее

In [5]:
tmp = MyContextMng('Hello')
f = tmp.__enter__()
try:   
    actions
    pass
except:
    pass
finally:
    tmp.__exit__(None, None, None)

Enter
Exit


Написание контекстных менеджеров можно упростить с помощью декоратора `@contextmanager` и генераторов

In [6]:
from contextlib import contextmanager

@contextmanager
def my_context(s):
    print('Enter')
    yield s
    print('Exit')
    
with my_context('Hello') as f:
    #actions
    pass    

Enter
Exit


## Операции с файлами

#### Чтение файла

In [7]:
with open('data/eur.csv', 'r', encoding='utf-8') as f:
    print(f.readline().strip())
    for c, line in enumerate(f):                        
        pass
        

2017-04-01,59.8107


In [8]:
with open('data/eur.csv', 'rb') as f:
    print(f.readline()[:20])
    print(f.readline().decode('utf-8')[:20])

b'2017-04-01,59.8107\n'
2017-04-04,59.8953



#### Список файлов в директории

In [9]:
import os

os.listdir('data/')

['currencies.csv',
 'eur.csv',
 'faces.npy',
 'lin_reg.txt',
 'man.png',
 'news.txt.gz',
 'texts.zip',
 'weather.csv']

In [10]:
os.path.join('data', 'texts.zip')

'data/texts.zip'

#### Временные файлы

Часто во время работы программы появляется необходимость сохранить какие-либо промежуточные данные во временный файл и потом его удалить. Для этих целей существуют два контекстных модуль  `tempfile`. Для примера, контекстный менеджер `tempfile.NamedTemporaryFile` создает именованный временный файл на входе (чаще всего в директории `/tmp` на Linux-системах и аналогичных директориях для временных файлов на других) и удаляет его на выходе. 

In [11]:
import tempfile

with tempfile.NamedTemporaryFile() as fn:
    with open(fn.name, 'w', encoding='utf-8') as f:
        f.write('Hello world!\n')
        
    with open(fn.name, 'r', encoding='utf-8') as f:
        print(f.readlines())

['Hello world!\n']


#### Чтение zip-архивов

Встроенный модуль `zipfile` позволяет читать файлы в `zip`-архивах, тем самым избегать явной распаковки.

In [12]:
import zipfile
import io

with zipfile.ZipFile('data/texts.zip', 'r') as zf:
    with zf.open('texts.txt', 'r') as f:
        f_unicode = io.TextIOWrapper(f, 'utf-8')
        print(f_unicode.readline()[:50])
        print(f.readline().decode('utf-8')[:50])

0	 «Школа злословия» учит прикусить язык Сохранитс
ют: приключенческая канва опиралась на отличное зн


## Коллекции и итераторы

#### Модуль [itertools](https://docs.python.org/3/library/itertools.html)

[itertools](https://docs.python.org/3/library/itertools.html) предоставляет различные утилиты для обработки коллекций и итераторов. 


Создание цепочки из итераторов

In [13]:
import itertools

def g():
    yield 1
    yield 2

ch = itertools.chain(g(), [1, 2, 3], {3, 5, 6}, iter([1, 2]))
list(ch)

[1, 2, 1, 2, 3, 3, 5, 6, 1, 2]

Группировка

In [14]:
for k, v in itertools.groupby('Hello World', key=lambda x: x.isupper()):
    print(k, list(v))

True ['H']
False ['e', 'l', 'l', 'o', ' ']
True ['W']
False ['o', 'r', 'l', 'd']


Кроме того имеет смысл помнить о наличии встроенных функций `zip` и `enumerate`, которые могут быть использованы для одновременного обхода нескольких коллекций и введения счетчика элементов соответственно.

In [15]:
list(zip(['а', 'б', 'в'], ['a', 'b', 'c']))

[('а', 'a'), ('б', 'b'), ('в', 'c')]

In [16]:
list(enumerate(['a', 'b', 'c']))

[(0, 'a'), (1, 'b'), (2, 'c')]

In [17]:
lst = ['a', 'b', 'c']
dict(enumerate(lst))

{0: 'a', 1: 'b', 2: 'c'}

#### Модуль [functools](https://docs.python.org/3/library/functools.html)

`functools` предоставляет различные утилиты для работы с функциями 

Например `partial` позволяет уменьшить число аргументов функции, создав новую функцию на основе старой

In [18]:
from functools import partial

add = lambda x, y: x + y
foo = partial(add, y=5)
foo(10)

15

`lru_cache` позволяет запоминать результат идемпотентной функции с помощью  [LRU](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_%D0%BA%D1%8D%D1%88%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F)-кэша

In [19]:
from functools import lru_cache
from time import sleep

@lru_cache(maxsize=5)
def heavy_stateless_computations(param):
    sleep(5)
    return param ** 2

%time heavy_stateless_computations(20)
%time heavy_stateless_computations(20)

CPU times: user 1.41 ms, sys: 239 µs, total: 1.65 ms
Wall time: 5.01 s
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


400

#### Модуль [collections](https://docs.python.org/3/library/collections.html)

Модуль предоставляет различные улучшения к стандартным коллекциям. Например, `defaultdict` позволяет создать словарь, в котором значение для новых ключей будет создаваться автоматически, как результат работы функции обратного вызова. 

In [20]:
from collections import defaultdict

d = defaultdict(lambda: []) # можно просто defaultdict(list)
d['word1'].append(1)
d['word1'].append(2)
d['word2'].append(3)
d

defaultdict(<function __main__.<lambda>()>, {'word1': [1, 2], 'word2': [3]})

`Counter` позволяет вести подсчет каких-то элементов.

In [21]:
from collections import Counter

c = Counter()
c['word1'] += 1
c['word2'] += 2
c.update({'word1': 5, 'word3': 4})
c

Counter({'word1': 6, 'word2': 2, 'word3': 4})

полезное свойство - получение наиболее частотных ключей

In [22]:
c.most_common(2)

[('word1', 6), ('word3', 4)]

In [23]:
from collections import OrderedDict

lst = [(5 - x, x) for x in range(5)]
d = dict(lst)
od = OrderedDict(lst)
print(list(d.keys()), list(od.keys()))

[5, 4, 3, 2, 1] [5, 4, 3, 2, 1]


## Сериализация

Сериализация - представление структуры данных в некотором виде виде, удобным для дальнейшего хранения этой структуры в файле или передачи (например по сети). В `Python` существует стандартный механизм сериализации - модуль `pickle`. Так же возможно сохранение базовых объектов в формате `json`. 

В данном примере сначала создаются временные файлы, куда записывается представление объектов в бинарном виде и в `json`. Затем это представление восстанавливается. 

In [24]:
import tempfile
import pickle
import json

obj = {'hello' : [1, 2, 3], 'world': [4, 5, 6]}

with tempfile.NamedTemporaryFile() as fn:
    with open(fn.name, 'wb') as f:
        pickle.dump(obj, f)
    print(pickle.load(open(fn.name, 'rb')))
    
with tempfile.NamedTemporaryFile() as fn:
    with open(fn.name, 'w') as f:
        json.dump(obj, f)
    print(json.load(open(fn.name, 'r')))    

{'hello': [1, 2, 3], 'world': [4, 5, 6]}
{'hello': [1, 2, 3], 'world': [4, 5, 6]}


Аналогичным образом можно сохранить, например, экземпляры `dataclass`'ов

In [25]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    year: int

persons = [Person("Ivan", "1975"), Person("Petr", "1985")]

serialized = pickle.dumps(persons)
recovered = pickle.loads(serialized)
recovered

[Person(name='Ivan', year='1975'), Person(name='Petr', year='1985')]